In [229]:
%load_ext autoreload
%autoreload 2

import pdfplumber
import re

from utils import create_new_sheet, write_content, format_sheet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [230]:
############ Update the following parameters ##################
FILEPATH = "../data/April statement.pdf"
NEW_SHEET_NAME = "June 2024"

In [231]:
SPREADSHEET_ID = "1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY"

In [232]:
pdf_lines = []
with pdfplumber.open(FILEPATH) as pdf:
    for page_id in range(len(pdf.pages)):
        page = pdf.pages[page_id]
        pdf_lines.extend(page.extract_text_lines())
pdf_lines = [line['text'] for line in pdf_lines]

In [233]:
regex_pattern = "^(\d\d/\d\d.*) (-*[\d\.,]+)$"
accumulated_content = [['Transaction', 'Cost', 'Type', 'Regular expenses', 'Trip expenses', 'Hobby expenses']]
row_id = 2
for line in pdf_lines:
    if "AUTOMATIC PAYMENT" in line:
        continue
    re_search = re.search(regex_pattern, line)
    if re_search is not None:
        cost = float(re_search.group(2))
        regular_cost = f'=IF(C{row_id}="Regular", B{row_id}, 0)'
        trip_cost = f'=IF(C{row_id}="Trip", B{row_id}, 0)'
        hobby_cost = f'=IF(C{row_id}="Hobby", B{row_id}, 0)'
        accumulated_content.append([re_search.group(1), cost, 'Regular', regular_cost, trip_cost, hobby_cost])
        row_id += 1
accumulated_content.append(['', f'=SUM(B2:B{row_id-1})', '', f'=SUM(D2:D{row_id-1})', f'=SUM(E2:E{row_id-1})', f'=SUM(F2:F{row_id-1})', f'=SUM(D{row_id}:F{row_id})'])

In [234]:
# Export to the spreadsheet.
create_new_sheet_response = create_new_sheet(SPREADSHEET_ID, NEW_SHEET_NAME)
sheet_id = create_new_sheet_response['replies'][0]['addSheet']['properties']['sheetId']
write_response = write_content(SPREADSHEET_ID, NEW_SHEET_NAME, accumulated_content)
format_sheet_response = format_sheet(SPREADSHEET_ID, sheet_id)

Created a new sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{'addSheet': {'properties': {'sheetId': 477129512, 'title': 'June 2024', 'index': 12, 'sheetType': 'GRID', 'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}}]}
337 cells updated.
Format sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{}, {}, {}]}


### TODO
- Classify the Regular, Trip and Hobby
- Sort the transactions by date